In [4]:
from dataclasses import dataclass
from typing import Union
import math

In [8]:
Node = Union[int, 'Pair']

@dataclass
class Pair:
    left: Node
    right: Node
        
def split(i):
    return Pair(i // 2, round(i / 2))
    
def explode(p):
    

In [13]:
split(Pair(11, 12))

Pair(left=11, right=12)

In [ ]:
xs = [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
build_tree(xs)

In [1]:
def build_tree(xs):
    pass